# **LEAP Runtime Test**
In this notebook, we will look at how fast LEAP is at executing some simple evolutionary algorithms.

This analysis will use a `ConstantProblem` where the fitness is always 1 no matter what the individual's genome is.

The bulk of the runtime in typical evolutionary algorithms (EAs) lie in fitness evaluation. Using a `ConstantProblem` avoids this computation altogether.

In [1]:
from leap_ec.algorithm import generational_ea
from leap_ec.problem import ConstantProblem
from leap_ec.representation import Representation
from leap_ec.real_rep.initializers import create_real_vector
from leap_ec.real_rep.ops import mutate_gaussian
from leap_ec.binary_rep.ops import mutate_bitflip
from leap_ec.binary_rep.initializers import create_binary_sequence
from leap_ec.binary_rep.problems import MaxOnes
from leap_ec.binary_rep.decoders import BinaryToIntDecoder
import leap_ec.ops as ops


## Define Hyperparameters
These parameters will be used to test various aspects of LEAP.

In [2]:
MAX_GENERATIONS = 100
POPULATION_SIZE = 10
GENOME_LENGTH = 10000
BOUNDS = [[-100, 100]]*GENOME_LENGTH

## Run Simple Real-valued EA
LEAP v0.6.0 Runtime: 30.955s

LEAP numpy Runtime: 0.877s

In [3]:
import time

start = time.time()
ea = generational_ea(max_generations=MAX_GENERATIONS, pop_size=POPULATION_SIZE,
                     problem=ConstantProblem(),
                     representation=Representation(
                         initialize=create_real_vector(bounds=BOUNDS)),
                     pipeline=[
                         ops.tournament_selection(k=2),
                         ops.clone,
                         ops.uniform_crossover,
                         mutate_gaussian(std=0.5, expected_num_mutations='isotropic'),
                         ops.evaluate,
                         ops.pool(size=POPULATION_SIZE)
                     ])
list(ea);
end = time.time()
print(f'Runtime: {end - start}')

Runtime: 30.95457911491394


## Run Simple Binary EA
LEAP v0.6.0 Runtime: 4.850s

LEAP numpy Runtime: 0.511s

Biggest difference in speedup is found in crossover and mutate bitflip since the genome is large

In [4]:
import time

start = time.time()
ea = generational_ea(max_generations=MAX_GENERATIONS, pop_size=POPULATION_SIZE,
                     problem=MaxOnes(),
                     representation=Representation(
                         initialize=create_binary_sequence(length=GENOME_LENGTH)),
                     pipeline=[
                         ops.tournament_selection(k=2),
                         ops.clone,
                         ops.uniform_crossover,
                         mutate_bitflip(probability=0.5),
                         ops.evaluate,
                         ops.pool(size=POPULATION_SIZE)
                     ])
list(ea);
end = time.time()
print(f'Runtime: {end - start}')

Runtime: 4.695421934127808


## Run Simple Integer-valued EA
LEAP v0.6.0 Runtime: s

LEAP numpy Runtime: s